In [ ]:
from sklearn.metrics import f1_score
# нужные библиотеки
import numpy as np
import pandas as pd
import gc
import random
import re
import typing
import numpy as np
import datetime
from time import sleep
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import mean_absolute_percentage_error, r2_score, mean_squared_error
from sklearn.metrics import roc_auc_score
import numpy as np
import optuna
import catboost as cb
from sklearn.datasets import load_breast_cancer
from sklearn.metrics import accuracy_score,recall_score
from sklearn.model_selection import train_test_split
from catboost import CatBoostClassifier
import ast
from collections.abc import Iterable

import pandas as pd
from sklearn.metrics import f1_score
from tqdm.auto import tqdm,trange
from lightautoml.automl.presets.tabular_presets import TabularAutoML, TabularUtilizedAutoML
from lightautoml.tasks import Task

import time

# LightAutoML presets, task and report generation
from lightautoml.automl.presets.tabular_presets import TabularAutoML, TabularUtilizedAutoML
from lightautoml.tasks import Task
from lightautoml.report.report_deco import ReportDeco

pd.set_option("display.max_rows", 100)
pd.set_option("display.max_columns", 100)
%matplotlib inline

%matplotlib inline

In [2]:
def ragged_flatten(xs):
    for x in xs:
        if isinstance(x, Iterable) and not isinstance(x, (str, bytes)):
            yield from ragged_flatten(x)
        else:
            yield x
            
def getLatLongitudeFromString(s):
    d = ast.literal_eval(s)
    if "geome" in s:
        l = d['geometries'][1]['coordinates']
        
    if "geome" not in s:
        l = d['coordinates']
        
    k = np.array(list(ragged_flatten(l)))
    
    x = np.mean(k[::2])
    y = np.mean(k[1::2])
    return (x,y,len(k)//2)

In [4]:
df = pd.read_csv("train_dataset.csv")
df.head()

,id,area,nd_mean_2021-04-16,nd_mean_2021-04-19,nd_mean_2021-04-22,nd_mean_2021-04-26,nd_mean_2021-04-28,nd_mean_2021-05-02,nd_mean_2021-05-04,nd_mean_2021-05-07,nd_mean_2021-05-16,nd_mean_2021-05-17,nd_mean_2021-05-21,nd_mean_2021-05-26,nd_mean_2021-05-27,nd_mean_2021-05-29,nd_mean_2021-06-06,nd_mean_2021-06-09,nd_mean_2021-06-13,nd_mean_2021-06-16,nd_mean_2021-06-18,nd_mean_2021-06-19,nd_mean_2021-06-20,nd_mean_2021-06-22,nd_mean_2021-06-25,nd_mean_2021-06-27,nd_mean_2021-07-07,nd_mean_2021-07-08,nd_mean_2021-07-09,nd_mean_2021-07-13,nd_mean_2021-07-15,nd_mean_2021-07-17,nd_mean_2021-07-20,nd_mean_2021-07-26,nd_mean_2021-07-27,nd_mean_2021-07-29,nd_mean_2021-07-31,nd_mean_2021-08-07,nd_mean_2021-08-12,.geo,nd_mean_2021-04-15,nd_mean_2021-04-18,nd_mean_2021-04-25,nd_mean_2021-04-27,nd_mean_2021-04-29,nd_mean_2021-04-30,nd_mean_2021-06-07,nd_mean_2021-06-12,nd_mean_2021-06-28,nd_mean_2021-08-01,nd_mean_2021-08-11,nd_mean_2021-05-01,nd_mean_2021-05-03,nd_mean_2021-05-10,nd_mean_2021-05-15,nd_mean_2021-06-02,nd_mean_2021-06-03,nd_mean_2021-07-04,nd_mean_2021-08-10,nd_mean_2021-08-23,nd_mean_2021-04-20,nd_mean_2021-04-23,nd_mean_2021-05-09,nd_mean_2021-05-19,nd_mean_2021-05-20,nd_mean_2021-06-04,nd_mean_2021-06-05,nd_mean_2021-06-10,nd_mean_2021-07-05,nd_mean_2021-08-13,nd_mean_2021-08-27,nd_mean_2021-05-08,nd_mean_2021-05-24,crop
0,3536,20,0.072846,0.261778,0.062981,0.104442,0.021096,0.052202,0.158723,0.000000,0.484102,0.201013,0.029708,0.397833,0.384805,0.015936,0.007275,0.441994,0.172777,0.000000,0.412072,0.507029,0.490741,0.549791,0.460996,0.100555,0.706968,0.691171,0.711135,0.731325,0.718093,0.000000,0.147169,0.728053,0.112933,0.727516,0.000000,0.000000,0.658016,"{""type"":""Polygon"",""coordinates"":[[[42.49107269...",0.187150,0.262300,0.077577,0.000476,0.000000,0.000000,0.111821,0.038902,0.155038,0.754859,0.000000,0.121799,0.083970,0.213006,0.358304,0.271128,0.173833,0.014060,0.030883,0.034858,0.063134,0.000000,0.102757,0.425571,0.108999,0.118854,0.000000,0.000000,0.026784,0.126832,0.614770,0.008857,0.081498,3
1,3739,45,0.332928,0.287182,0.097138,0.152467,0.075484,0.000000,0.140326,0.163340,0.519456,0.501486,0.047332,0.448796,0.692873,0.628463,0.016747,0.000000,0.059826,0.000000,0.492250,0.708562,0.668011,0.675501,0.694017,0.654727,0.709037,0.696734,0.268523,0.076741,0.638567,0.676917,0.460940,0.059759,0.073804,0.666918,0.089721,0.184231,0.601685,"{""type"":""Polygon"",""coordinates"":[[[50.25545110...",0.245285,0.349133,0.173578,0.196361,0.015048,0.242130,0.708402,0.150383,0.432990,0.706226,0.618475,0.047408,0.000000,0.160796,0.031104,0.000000,0.040128,0.194295,0.062582,0.000000,0.064671,0.024539,0.000000,0.248935,0.134469,0.208268,0.000000,0.000000,0.111148,0.174914,0.179612,0.113071,0.046997,4
2,1294,28,0.013679,0.007055,0.120804,0.037839,0.259125,0.000000,0.037446,0.506516,0.000000,0.561541,0.000000,0.029533,0.245910,0.650065,0.000000,0.516493,0.000000,0.717313,0.649345,0.740904,0.652131,0.739836,0.595076,0.741171,0.648106,0.241615,0.586071,0.551204,0.522871,0.543462,0.000000,0.172532,0.119380,0.321827,0.411168,0.357296,0.000000,"{""type"":""Polygon"",""coordinates"":[[[37.99698998...",0.006391,0.000000,0.041751,0.169020,0.011260,0.118255,0.056252,0.011727,0.172335,0.000000,0.172080,0.230169,0.024845,0.151504,0.002801,0.011655,0.000000,0.135650,0.045370,0.000000,0.000000,0.112471,0.028961,0.127633,0.084467,0.000000,0.210496,0.130467,0.155225,0.090607,0.054127,0.007437,0.219614,2
3,2859,19,0.105976,0.052500,0.001917,0.011531,0.042316,0.000000,0.000000,0.043604,0.510271,0.511552,0.000000,0.524763,0.707093,0.645025,0.753284,0.000000,0.032089,0.064643,0.543500,0.706039,0.617195,0.712928,0.628014,0.563310,0.591204,0.557154,0.188446,0.466411,0.161966,0.425800,0.094829,0.474245,0.242224,0.000000,0.055983,0.060656,0.327025,"{""type"":""Polygon"",""coordinates"":[[[44.72154181...",0.293425,0.318047,0.072589,0.000000,0.145339,0.007611,0.008742,0.099960,0.229617,0.152193,0.379196,0.046483,0.063252,0.099087,0.412416,0.441948,0.7471

In [5]:
df['mean_latitude'] = [getLatLongitudeFromString(str(s['.geo']))[0] for _,s in tqdm(df.iterrows())]
df['mean_longitude'] = [getLatLongitudeFromString(str(s['.geo']))[1] for _,s in tqdm(df.iterrows())]
#df['edge_num'] = [getLatLongitudeFromString(str(s['.geo']))[2] for _,s in tqdm(df.iterrows())]
#df['log_area'] = np.log10(df['area'])

In [6]:
df_geo = df[['.geo','mean_latitude','mean_longitude']].copy()

In [8]:
import meteostat

In [19]:
from datetime import datetime
import matplotlib.pyplot as plt
from meteostat import Point, Daily,Monthly
import time
# Set time period
start = datetime(2021, 4, 15)
end = datetime(2021, 8, 12)

# Create Point for Vancouver, BC
vancouver = Point(49.2497, -123.1193, 70)

# Get daily data for 2018
data = Daily(vancouver, start, end)
data = data.fetch()

In [ ]:
round(df_geo['mean_longitude'][i],2)

In [11]:
datas = []
for i in trange(df_geo.shape[0]):
    coord = Point(round(df_geo['mean_longitude'][i],1),round(df_geo['mean_latitude'][i],1),10)
    data = Monthly(coord, start, end)
    data = data.fetch()
    datas.append(data)
    time.sleep(0.2)

In [18]:
coord = Point(round(df_geo['mean_longitude'][i]),round(df_geo['mean_latitude'][i]))
data = Monthly(coord, start, end)
data.fetch()


,tavg,tmin,tmax,prcp,wspd,pres,tsun


In [ ]:
datas[0]

In [ ]:
import joblib
joblib.dump(datas, 'train_weather_1.joblib')  

In [21]:
df_test = pd.read_csv("test_dataset.csv")
df_test['mean_latitude'] = [getLatLongitudeFromString(str(s['.geo']))[0] for _,s in df_test.iterrows()]
df_test['mean_longitude'] = [getLatLongitudeFromString(str(s['.geo']))[1] for _,s in df_test.iterrows()]
##df_test['edge_num'] = [getLatLongitudeFromString(str(s['.geo']))[2] for _,s in df_test.iterrows()]
#df_test['log_area'] = np.log10(df_test['area'])


In [22]:
datas_test = []
for i in trange(df_test.shape[0]):
    coord = Point(round(df_test['mean_longitude'][i]),round(df_test['mean_latitude'][i]))
    data_test = Monthly(coord, start, end)
    data_test = data_test.fetch()
    datas_test.append(data_test)
    time.sleep(0.1)
datas_test

[            tavg  tmin  tmax  prcp  wspd    pres  tsun
 time                                                  
 2021-04-01   6.9   1.9  11.6  67.4  13.0  1015.0   NaN
 2021-05-01  14.1   9.2  18.5  71.2  14.6  1012.7   NaN
 2021-06-01  20.0  15.0  24.6  59.5  11.8  1013.8   NaN
 2021-07-01  23.1  17.7  27.9  61.2   9.9  1013.3   NaN
 2021-08-01  22.1  17.0  27.0  59.5  11.1  1013.4   NaN,
 Empty DataFrame
 Columns: [tavg, tmin, tmax, prcp, wspd, pres, tsun]
 Index: [],
 Empty DataFrame
 Columns: [tavg, tmin, tmax, prcp, wspd, pres, tsun]
 Index: [],
 Empty DataFrame
 Columns: [tavg, tmin, tmax, prcp, wspd, pres, tsun]
 Index: [],
             tavg  tmin  tmax  prcp  wspd    pres  tsun
 time                                                  
 2021-04-01   6.8   1.6  11.6  31.3  15.9  1016.6   NaN
 2021-05-01  15.8   9.8  21.4  80.2  13.4  1014.0   NaN
 2021-06-01  20.6  14.9  25.9  41.3  10.7  1015.4   NaN
 2021-07-01  21.5  15.3  27.3  54.0  10.2  1012.3   NaN
 2021-08-01  21.7  15.4  

In [ ]:
datas_test[:10]

In [24]:
import joblib
joblib.dump(datas_test, 'test_weather_1.joblib')  

['test_weather_1.joblib']

In [ ]:
round(df_test['mean_longitude'][i],2)